# Experiment 01: Airline dataset

In this experiment we use [the airline dataset](http://kt.ijs.si/elena_ikonomovska/data.html) to predict arrival delay. The dataset consists of a large amount of records, containing flight arrival and departure details for all the commercial flights within the USA, from October 1987 to April 2008. Its size is around 116 million records and 5.76 GB of memory.



In [18]:
import os,sys
import numpy as np
import pandas as pd
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (confusion_matrix, accuracy_score, roc_auc_score, f1_score, log_loss, precision_score,
                             recall_score, mean_squared_error, mean_absolute_error, r2_score)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from libs.loaders import load_airline
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric

os.environ['MOUNT_POINT'] = '/strata'
print("System version: {}".format(sys.version))
print("XGBoost version: {}".format(xgboost.__version__))
print("LightGBM version: {}".format(lightgbm.__version__))


System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


# 1) XGBoost vs LightGBM benchmark
In the next section we compare both libraries speed, accuracy and other metrics for the dataset of airline arrival delay. 

### Data loading and management

In [19]:
%%time
df_plane = load_airline()
print(df_plane.shape)

(115069017, 14)
CPU times: user 1min 42s, sys: 13.2 s, total: 1min 55s
Wall time: 4min 11s


In [20]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


The first step is to convert the categorical features to numeric features.

In [21]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])


CPU times: user 1min 51s, sys: 17.3 s, total: 2min 8s
Wall time: 2min 10s


In [22]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [23]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')


CPU times: user 59.8 s, sys: 13.2 s, total: 1min 12s
Wall time: 1min 13s


In [24]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


To simplify the pipeline, we are going to set a classification problem where the goal is to classify wheather a flight has arrived delayed or not. For that we need to binarize the variable `ArrDelay`.

If you want to extend this experiment, you can set a regression problem and try to identify the number of minutes of delay a fight has. Both XGBoost and LightGBM have regression classes.

In [25]:
%%time
filter_func = lambda x: 1 if x > 0 else 0
df_plane_numeric['ArrDelayBinary'] = df_plane_numeric['ArrDelay'].map(filter_func)

CPU times: user 38.7 s, sys: 7.5 s, total: 46.2 s
Wall time: 46.8 s


In [26]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,ArrDelayBinary
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27,1
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5,1
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17,1
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2,0
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17,1


Once the features are prepared, let's split the dataset into train, validation and test set.

In [27]:
def split_train_val_test_df(df, val_size=0.2, test_size=0.2):
    train, validate, test = np.split(df.sample(frac=1), 
                                     [int((1-val_size-test_size)*len(df)), int((1-test_size)*len(df))])
    return train, validate, test

In [28]:
%%time
train, validate, test = split_train_val_test_df(df_plane_numeric)
print(train.shape)
print(validate.shape)
print(test.shape)

(69041410, 15)
(23013803, 15)
(23013804, 15)
CPU times: user 44.2 s, sys: 39.1 s, total: 1min 23s
Wall time: 1min 24s


In [ ]:
def generate_feables(df):
    X = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])]
    y = df['ArrDelayBinary']
    return X,y

In [29]:
%%time
X_train, y_train = generate_feables(train)
X_val, y_val = generate_feables(validate)
X_test, y_test = generate_feables(test)


CPU times: user 2.08 s, sys: 2.94 s, total: 5.02 s
Wall time: 5.09 s


### Training 
Now we are going to create two pipelines, one of XGBoost and one for LightGBM. The technology behind both libraries is different, so it is difficult to compare them in the exact same model setting. XGBoost grows the trees depth-wise and controls model complexity with `max_depth`. Instead, LightGBM uses a leaf-wise algorithm and controls the model complexity by `num_leaves`. As a tradeoff, we use XGBoost with `max_depth=8`, which will have max number leaves of 255, and compare it with LightGBM with `num_leaves=255`. 

Let's start with the XGBoost classifier.

In [30]:
pipeline_steps = [('scale', StandardScaler())]
continuous_pipeline = Pipeline(steps=pipeline_steps)
featurisers = [('continuous', continuous_pipeline)]

In [33]:
xgb_clf_pipeline = Pipeline(steps=[('features', FeatureUnion(featurisers)),
                                 ('clf', XGBClassifier(max_depth=8,
                                                      n_estimators=100,
                                                      min_child_weight=30,
                                                      learning_rate=0.1,
                                                      subsample=0.80,
                                                      colsample_bytree=0.80,
                                                      seed=77))])
    
xgb_clf_pipeline = XGBClassifier(max_depth=8,
                                n_estimators=100,
                                min_child_weight=30,
                                learning_rate=0.1,
                                subsample=0.80,
                                colsample_bytree=0.80,
                                seed=77)

In [ ]:
%%time
#xgb_clf_pipeline.fit(X_train, y_train, clf__eval_set=[(X_val, y_val)], clf__verbose=True, clf__eval_metric='rmse')
xgb_clf_pipeline.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True, eval_metric='rmse')

[0]	validation_0-rmse:0.493304
[1]	validation_0-rmse:0.488668
[2]	validation_0-rmse:0.483633
[3]	validation_0-rmse:0.481827
[4]	validation_0-rmse:0.4784
[5]	validation_0-rmse:0.477204
[6]	validation_0-rmse:0.474075
[7]	validation_0-rmse:0.471211
[8]	validation_0-rmse:0.470229
[9]	validation_0-rmse:0.469272
[10]	validation_0-rmse:0.468569
[11]	validation_0-rmse:0.467859
[12]	validation_0-rmse:0.465366
[13]	validation_0-rmse:0.463309
[14]	validation_0-rmse:0.461693
[15]	validation_0-rmse:0.459783
[16]	validation_0-rmse:0.459253
[17]	validation_0-rmse:0.457811
[18]	validation_0-rmse:0.45683
[19]	validation_0-rmse:0.456078
[20]	validation_0-rmse:0.455063
[21]	validation_0-rmse:0.454124
[22]	validation_0-rmse:0.453167
[23]	validation_0-rmse:0.45221
[24]	validation_0-rmse:0.451672
[25]	validation_0-rmse:0.450589
[26]	validation_0-rmse:0.450187
[27]	validation_0-rmse:0.449189
[28]	validation_0-rmse:0.448456
[29]	validation_0-rmse:0.447432
[30]	validation_0-rmse:0.447029


Training LightGBM classifier

In [ ]:
lgbm_clf_pipeline = LGBMClassifier(num_leaves=255,
                                    n_estimators=100,
                                    min_child_weight=30,
                                    learning_rate=0.1,
                                    nthread=20,
                                    subsample=0.80,
                                    colsample_bytree=0.80,
                                    seed=77)

In [ ]:
%%time
lgbm_clf_pipeline.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True, eval_metric='rmse')

As it can be seen in the results, given the specific versions of both XGBoost and LightGBM and in this specific dataset, LightGBM is faster. 

In general terms, leaf-wise algorithms are more efficient, they converge much faster than depth-wise. However, it may cause over-fitting when the data is small or there are too many leaves.

### Evaluation
Now let's evaluate the model in the test set.

In [ ]:
%%time
y_prob_xgb = xgb_clf_pipeline.predict_proba(X_test)

In [ ]:
%%time
y_prob_lgbm = lgbm_clf_pipeline.predict_proba(X_test)

### Metrics
We are going to obtain some metrics to evaluate the performance of each of the models.

In [ ]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary(y_true, y_pred):
    m_acc = accuracy_score(y_true, y_pred)
    m_f1 = f1_score(y_true, y_pred)
    m_precision = precision_score(y_true, y_pred)
    m_recall = recall_score(y_true, y_pred)
    m_conf = confusion_matrix(y_true, y_pred)
    report = {'Accuracy':m_acc, 'Precision':m_precision, 'Recall':m_recall, 'F1':m_f1, 'Confusion Matrix':m_conf}
    return report

In [ ]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary_prob(y_true, y_prob):
    m_auc = roc_auc_score(y_true, y_prob)
    m_logloss = log_loss(y_true, y_prob)
    report = {'AUC':m_auc, 'Log loss':m_logloss}
    return report

In [ ]:
def binarize_prediction(y, threshold=0.5):
    threshold_func = lambda x: 0 if x<=threshold else 1
    y_pred = y.map(threshold_func)
    return y_pred


In [ ]:
y_pred_xgb = binarize_prediction(y_prob_xgb)
y_pred_lgbm = binarize_prediction(y_prob_lgbm)


In [ ]:
report1_xgb = classification_metrics_binary(y_test, y_pred_xgb)
print(report1_xgb)
report2_xgb = classification_metrics_binary_prob(y_test, y_prob_xgb)
print(report2_xgb)

In [ ]:
report1_lgbm = classification_metrics_binary(y_test, y_pred_lgbm)
print(report1_lgbm)
report2_lgbm = classification_metrics_binary_prob(y_test, y_prob_lgbm)
print(report2_lgbm)

The experiment shows a fairly similar performance in both libraries, being LightGBM slightly better.

# 2) Concept drift
In this section we are trying to find concept drift in the dataset to check if retraining is valuable.

### Data management
We are going to pack the data yearly to try to find concept drift

In [ ]:
def get_data_list_yearly(df):
    data_yearly = [df[df['Year'] == year] for year in range(1987, 2008)]  
    return data_yearly

In [ ]:
%%time
data_yearly_list = get_data_list_yearly(df_plane_numeric)
for subset in data_yearly_list:
    print(subset.shape)

In [ ]:
total_subsets = len(data_yearly_list)
print("Number of years: {}".format(total_subsets))
num_ini = 5

In [ ]:
def generate_subset(data_yearly_list, num):
    subset = data_yearly_list[0]
    for i in range(1,num):
        subset = pd.concat([subset, data_yearly_list[i]])
    return subset

### Traininig
Let's see what happens when we train on a subset of data and then evaluate in the data of the following years.

In [ ]:
%%time
subset_base = generate_subset(data_yearly_list, num_ini)
print(subset_base.shape)

In [ ]:
X_train, y_train = generate_feables(subset_base)

In [ ]:
clf = LGBMClassifier(num_leaves=255,
                    n_estimators=100,
                    min_child_weight=30,
                    learning_rate=0.1,
                    subsample=0.80,
                    colsample_bytree=0.80,
                    seed=42)

In [ ]:
%%time
clf.fit(X_train, y_train)

In [ ]:
def predict_accuracy_future_years(clf, data_yearly_list, num_ini):
    total_subsets = len(data_yearly_list)
    accuracy_dict = {}
    for y in range(num_ini, total_subsets):
        year = data_yearly_list[y]['Year'].iloc[0]
        print("Predicting year {}...".format(year))
        X_test, y_test = generate_feables(data_yearly_list[y])
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        accuracy_dict[year] = acc
    return accuracy_dict

In [ ]:
%%time
accuracy_dict = predict_accuracy_future_years(clf, data_yearly_list, num_ini)
print(accuracy_dict)

From the results we can observe that the accuracy of the model gets worse as the years pass on.

### Retraining
Now let's see what happens when we retrain and evaluate in the data of the following years.

In [ ]:
new_init = 15

In [ ]:
%%time
subset_retrain = generate_subset(data_yearly_list, new_init)
print(subset_retrain.shape)

In [ ]:
X_train, y_train = generate_feables(subset_retrain)

In [ ]:
clf_retrain = LGBMClassifier(num_leaves=255,
                    n_estimators=100,
                    min_child_weight=30,
                    learning_rate=0.1,
                    subsample=0.80,
                    colsample_bytree=0.80,
                    seed=42)

In [ ]:
%%time
clf_retrain.fit(X_train, y_train)

In [ ]:
%%time
accuracy_retrain = predict_accuracy_future_years(clf_retrain, data_yearly_list, new_init)
print(accuracy_retrain)

As it can be seen, the performance is better after retraining. We have found concept drift in this dataset.